# Setup - Load an API Key and Libraries

## Storing and Loading Your API Key Securely

To securely store and use your API key in this notebook, follow these steps:

1. **Generate an API key**:
   - Follow instructions in the Getting Started documentation for Hub APIs to generate a Hub API Key.

2. **Create a `.env` File**:
   - In the same directory as this notebook, create a file named `.env`.
   - Add the following line to the file, replacing `<your_api_key>` with your actual API key:
     ```plaintext
     API_KEY=<your_api_key>
     ```
   - Save the file.

3. **Load the Key in the Notebook**:
   - The following code snippet is already included in this notebook to load the key securely:
     ```python
     import os
     from dotenv import load_dotenv
     
     load_dotenv(".env")  # Ensure the path matches your `.env` file location
     key = os.getenv("API_KEY")
     ```
   - This will load the `API_KEY` from your `.env` file into the `key` variable.

4. **Keep the `.env` File Secure**:
   - Do not share your `.env` file or API key.
   - If you do accidentally share your key, delete it in the workspaces UI and request a new one.

By following these steps, you can securely store and use your API key without exposing it in the notebook.

## Setting Up the Workspace and Environment

To ensure the notebook works correctly, you need to configure the `workspace` and `environment` variables. Follow these steps:
1. **Ensure your workspace is configured for commercial orders**
   - Follow instructions in the Hub documentation under `Data - Commercial` to link your commercial account to your workspace.
2. **Set the `workspace` Variable**:
   - In the following cell, the `workspace` variable should be set to the name of the workspace you wish to order data for.
   - You can find the workspace name in the site UI.
3. **Set the `environment` Variable**
   - In the following cell, The environment variable determines which environment you are working in. It can only be one of the following values:
     - prod (Production)
     - staging (Staging)
     - test (Testing)
     - dev (Development)

In [ ]:
import os
from dotenv import load_dotenv

workspace = "apalmer-tpzuk"
environment = "prod"

load_dotenv(".env")
api_key = os.getenv("API_KEY")

if environment == "prod":
    platform_domain = "https://prod.eodatahub.org.uk"
elif environment == "staging":
    platform_domain = "https://staging.eodatahub.org.uk"
elif environment == "test":
    platform_domain = "https://test.eodatahub.org.uk"
elif environment == "dev":
    platform_domain = "https://dev.eodatahub.org.uk"
else:
    raise ValueError("Invalid environment. Choose from 'prod', 'staging', 'test', or 'dev'.")

In [ ]:
!pip install pystac-client xarray rasterio

# Create a pystac-client Client for the EODH Catalogue

The EODH catalogue can be viewed through a UI on the Hub under the `Catalogue` tab.

Our STAC catalogue can also be browsed programatically using tools such as pystac-client, using your api_key for authorisation. In the following example, our client is scoped to the Planet catalogue to refine a search for some Planet commercial data.

In [ ]:
from pystac_client import Client

# Limit scope of the top-level catalogue to Planet
rc_url = f"{platform_domain}/api/catalogue/stac/catalogs/commercial/catalogs/planet"

# Create STAC client
stac_client = Client.open(rc_url, headers={"Authorization": f"Bearer {api_key}"})

# Search for Planet Data

To demonstrate search capabilities, the following cell searches our catalogue for PSScene items that intersect a given polygon AOI.

In [ ]:
geom = {
    "type": "Polygon",
    "coordinates": [
        [
            [9.6, 57.1],
            [9.6, 57.0],
            [9.8, 56.9],
            [9.8, 57.0],
            [9.6, 57.1],
        ]
    ],
}
search = stac_client.search(
    max_items=10,
    collections=['PSScene'],
    intersects=geom,
)
for item in search.items():
    print(item.get_self_href())

# Obtain a Quote for Planet Data

Once an item of interest is found, you may obtain a quote for the item via a POST request to /quote following the item href.

Change the `item_href` to any valid link to a STAC item in our Planet commercial catalogue to browse prices. Change the `coordinates` to an AOI to clip the item if required.

Note: Coordinates are to be provided in longitude/latitude WGS84, and the polygon must be closed. Coordinates follow OGC GeoJSON specification.

Note: Planet quotes are given in km<sup>2</sup>.

In [ ]:
import requests

item_href = "https://staging.eodatahub.org.uk/api/catalogue/stac/catalogs/commercial/catalogs/planet/collections/PSScene/items/20250217_101155_07_24c7"
url = f"{item_href}/quote"
headers = {
    'accept': 'application/json', 
    'Content-Type': 'application/json', 
    'Authorization': f'Bearer {api_key}'
}
data =  {
    "coordinates": [
        [
            [9.6, 57.1],
            [9.6, 57.0],
            [9.8, 56.9],
            [9.8, 57.0],
            [9.6, 57.1]
        ]
    ]
}

response = requests.post(url, headers=headers, json=data)

print("Status Code", response.status_code)
print("Response ", response.json())

# Order Planet Data

Once you are satisfied with the item and pricing, you may proceed with ordering the item via a POST request to /order following the item href.

Change the `item_href` to any valid link to a STAC item in our Planet commercial catalogue to order it. Change the `coordinates` to an AOI to clip the item if required.
The `productBundle` field may also be changed to adjust the processing done on the image. For details please see our guide on ordering commercial data on the Hub.

Once an order is placed, the API response includes a Location header that links to the item in your workspace catalogue. This item will be updated with the order status and assets when they are ready.

An order cannot be changed once it is placed, so please ensure the correct product and parameters are chosen.

In [ ]:
import requests

item_href = "https://staging.eodatahub.org.uk/api/catalogue/stac/catalogs/commercial/catalogs/planet/collections/PSScene/items/20250217_101155_07_24c7"
url = f"{item_href}/order"
headers = {
    'accept': 'application/json', 
    'Content-Type': 'application/json', 
    'Authorization': f'Bearer {api_key}'
}
data =  {
    "productBundle": "General use",
    "coordinates": [
        [
            [9.6, 57.1],
            [9.6, 57.0],
            [9.8, 56.9],
            [9.8, 57.0],
            [9.6, 57.1]
        ]
    ]
}

response = requests.post(url, headers=headers, json=data)

print("Status Code:", response.status_code)
print("Response:", response.json())

location_header = response.headers.get('Location')
print("Item will shortly be available at:", location_header)

# Read STAC for Ordered Planet Data

The STAC item can also be obtained through browsing your workspace catalogue. The following example demonstrates a search for a specific asset for an item.

In [ ]:
from pystac import Item
from pystac_client import CollectionClient

data_i_ordered_earlier = f"{platform_domain}/api/catalogue/stac/catalogs/user/catalogs/{workspace}/catalogs/commercial-data/catalogs/planet"
stac_client = Client.open(data_i_ordered_earlier, headers={"Authorization": f"Bearer {api_key}"})

In [ ]:
ordered_item = next(stac_client.get_items("20250217_101155_07_24c7"))
ordered_item

In [ ]:
asset = ordered_item.get_assets()["20250217_101155_07_24c7_3B_AnalyticMS_clip.tif"]
asset

# Fetch Ordered Planet Data

Any asset link from a commercial data purchase can be downloaded via an authorised GET request to the asset href.

In [ ]:
import urllib3
from io import BytesIO

resp = urllib3.request("GET", asset.href, headers={"Authorization": f"Bearer {api_key}"})
resp.data[0:100]

# Plot Planet Data

Visual assets can be viewed programmatically:

In [ ]:
from rasterio.plot import show
import rasterio

with rasterio.open(BytesIO(resp.data)) as src:
    f = src.read()
    fig, ax = plt.subplots(figsize=(10, 10))
    show(f / 10000, ax=ax, title="PlanetScope Image")
    plt.show()

# Further Visualisation of Planet Data

Our TiTiler instance can also be used to visualise the data:

In [ ]:
import requests
from IPython.display import Image, display

TITILER_PREVIEW_URL = f'{platform_domain}/titiler/core/cog/preview'
TITILER_PREVIEW_PARAMS = {
    'url': asset.href,
    'bidx': 1,
    'rescale': '2229,12331',
    'colormap_name': 'rain_r'
}

response = requests.get(TITILER_PREVIEW_URL, params=TITILER_PREVIEW_PARAMS, headers={'Authorization': f'Bearer {api_key}'})

# Display the image
image = Image(response.content)
display(image)

# Obtain a Quote for Airbus SAR Data

Search and ordering capabilities are also in place for Airbus SAR items. Once an item of interest is found, you may obtain a quote for the item via a POST request to /quote following the item href.

Change the `item_href` to any valid link to a STAC item in our Airbus commercial catalogue to browse prices. Change the `licence` to another supported licence if required. Supported licences are `Single User Licence`, `Multi User (2 - 5) Licence`, and `Multi User (6 - 30) Licence`.

Note: Airbus SAR does not support clipping an item with an AOI.

In [ ]:
import requests

item_href = "https://staging.eodatahub.org.uk/api/catalogue/stac/catalogs/commercial/catalogs/airbus/collections/airbus_sar_data/items/TSX-1_WS_S_wide_001R_97985_D33003943_29000"
url = f"{item_href}/quote"
headers = {
    'accept': 'application/json', 
    'Content-Type': 'application/json', 
    'Authorization': f'Bearer {api_key}'
}
data =  {
    "licence": "Single User Licence",
}

response = requests.post(url, headers=headers, json=data)

print("Status Code:", response.status_code)
print("Response:", response.json())

# Order Airbus SAR Data

Once you are satisfied with the item and pricing, you may proceed with ordering the item via a POST request to /order following the item href.

Change the `item_href` to any valid link to a STAC item in our Airbus commercial catalogue to order it. Change the `licence` to another supported licence if required. Supported licences are `Single User Licence`, `Multi User (2 - 5) Licence`, and `Multi User (6 - 30) Licence`.

The `productBundle` and `radarOptions` fields may also be changed to adjust the processing done on the image. Valid product bundles for SAR are `SSC`, `MGD`, `GEC`, and `EEC`. `radarOptions` include:
  - `orbit` of `rapid` or `science`.
  - `resolutionVariant` of `RE` or `SE`.
  - `projection` of `Auto`, `UTM`, or `UPS`.
  
For further details please see Airbus documentation.

Once an order is placed, the API response includes a Location header that links to the item in your workspace catalogue. This item will be updated with the order status and assets when they are ready.

An order cannot be changed once it is placed, so please ensure the correct product and parameters are chosen.

In [ ]:
import requests

item_href = "https://staging.eodatahub.org.uk/api/catalogue/stac/catalogs/commercial/catalogs/airbus/collections/airbus_sar_data/items/TSX-1_WS_S_wide_001R_97985_D33003943_29000"
url = f"{item_href}/order"
headers = {
    "accept": "application/json", 
    "Content-Type": "application/json", 
    "Authorization": f"Bearer {api_key}"
}
data =  {
    "licence": "Single User Licence",
    "productBundle": "SSC",
    "radarOptions": {
        "orbit": "rapid"
    }
}

response = requests.post(url, headers=headers, json=data)

print("Status Code", response.status_code)
print("Response ", response.json())

location_header = response.headers.get('Location')
print("Item will shortly be available at:", location_header)

# Obtain a Quote for Airbus Optical Data

Search and ordering capabilities are also in place for Airbus Optical items. Once an item of interest is found, you may obtain a quote for the item via a POST request to /quote following the item href.

Change the `item_href` to any valid link to a STAC item in our Airbus commercial catalogue to browse prices. Change the `coordinates` to an AOI to clip the item if required.

Note: Coordinates are to be provided in longitude/latitude WGS84, and the polygon must be closed. Coordinates follow OGC GeoJSON specification.

Change the `licence` to another supported licence if required. Supported licences are:
  - `Standard`
  - `Background Layer`
  - `Standard + Background Layer`
  - `Academic`
  - `Media Licence`
  - `Standard Multi End-Users (2-5)`
  - `Standard Multi End-Users (6-10)`
  - `Standard Multi End-Users (11-30)`
  - `Standard Multi End-Users (>30)`

In [ ]:
import requests

item_href = "https://staging.eodatahub.org.uk/api/catalogue/stac/catalogs/commercial/catalogs/airbus/collections/airbus_phr_data/items/DS_PHR1A_201203021558128_FR1_PX_W080S03_0221_01728"
url = f"{item_href}/quote"
headers = {
    'accept': 'application/json', 
    'Content-Type': 'application/json', 
    'Authorization': f'Bearer {api_key}'
}
data =  {
    "licence": "Standard",
    "coordinates": [
        [
            [-79.8,-2.1], 
            [-79.8,-2.2], 
            [-79.95,-2.2], 
            [-79.95,-2.1], 
            [-79.8,-2.1]
        ]
    ]
}

response = requests.post(url, headers=headers, json=data)

print("Status Code", response.status_code)
print("Response ", response.json())

# Order Airbus Optical Data

Once you are satisfied with the item and pricing, you may proceed with ordering the item via a POST request to /order following the item href.

- Change the `item_href` to any valid link to a STAC item in our Airbus commercial catalogue to order it. 
- `coordinates` can optionally clip an item to an AOI.
- `licence` must be set depending on the intended usage of the data. Supported licences are:
  - `Standard`
  - `Background Layer`
  - `Standard + Background Layer`
  - `Academic`
  - `Media Licence`
  - `Standard Multi End-Users (2-5)`
  - `Standard Multi End-Users (6-10)`
  - `Standard Multi End-Users (11-30)`
  - `Standard Multi End-Users (>30)`
- Set the `productBundle` field to adjust the processing done on the image. For details please see our guide on ordering commercial data on the Hub.
- `endUserCountry` must match an Airbus country code corresponding to the user making the purchase.

Once an order is placed, the API response includes a Location header that links to the item in your workspace catalogue. This item will be updated with the order status and assets when they are ready.

An order cannot be changed once it is placed, so please ensure the correct product and parameters are chosen.

In [ ]:
import requests

item_href = "https://staging.eodatahub.org.uk/api/catalogue/stac/catalogs/commercial/catalogs/airbus/collections/airbus_phr_data/items/DS_PHR1A_201203021558128_FR1_PX_W080S03_0221_01728"
url = f"{item_href}/order"
headers = {
    'accept': 'application/json', 
    'Content-Type': 'application/json', 
    'Authorization': f'Bearer {api_key}'
}
data =  {
    "licence": "Standard",
    "endUserCountry": "GB",
    "productBundle": "General use", 
    "coordinates": [
        [
            [-79.8,-2.1], 
            [-79.8,-2.2], 
            [-79.95,-2.2], 
            [-79.95,-2.1], 
            [-79.8,-2.1]
        ]
    ]
}

response = requests.post(url, headers=headers, json=data)

print("Status Code:", response.status_code)
print("Response:", response.json())

location_header = response.headers.get('Location')
print("Item will shortly be available at:", location_header)